In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from joblib import dump

In [2]:
data_dir = r"imagens\indie\pi\dataset\v20220930"
img_size = 16

X, y = [], []

for root, dirs, files in os.walk(data_dir):
    pasta_vogal = os.path.basename(os.path.dirname(root))
    letra = pasta_vogal[0].lower() if pasta_vogal else None
    if letra in ['a','e','i','o','u']:
        for file in files:
            if file.lower().endswith(('.png','.jpg','.jpeg')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (img_size, img_size))
                X.append(img.flatten())
                y.append(1 if letra == 'i' else 0)

X = np.array(X)
y = np.array(y)
print("Total de imagens carregadas:", len(X))



Total de imagens carregadas: 10000


In [3]:
# Configurações
n_splits = 3
random_state = 42

def evaluate_model(model, X, y, kf):
    accs, precs, recs, f1s = [], [], [], []
    for train_idx, test_idx in kf.split(X, y):
        model.fit(X[train_idx], y[train_idx])
        preds = model.predict(X[test_idx])
        accs.append(accuracy_score(y[test_idx], preds))
        precs.append(precision_score(y[test_idx], preds))
        recs.append(recall_score(y[test_idx], preds))
        f1s.append(f1_score(y[test_idx], preds))
    return np.mean(accs), np.mean(precs), np.mean(recs), np.mean(f1s)

kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

# Criar modelos (removendo SVM)
model_knn = KNeighborsClassifier(n_neighbors=3)
model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200)

# Avaliar modelos
results = {}
for name, model in [("KNN", model_knn), ("MLP", model_mlp)]:
    print(f"Treinando e avaliando {name}...")
    acc, prec, rec, f1 = evaluate_model(model, X, y, kf)
    results[name] = (acc, prec, rec, f1)
    # Salvar modelo final
    dump(model, f"{name}_final_model.pkl")

# Exibir resultados
df = pd.DataFrame(results, index=["Acurácia", "Precision", "Recall", "F1"]).T
print(df)

# -----------------------------
# Salvar resultados para futuras execuções
# -----------------------------

# Salvar em NPZ (Python)
np.savez("avaliacao_modelos.npz", **results)

# Salvar em CSV (legível)
df.to_csv("avaliacao_modelos.csv")

print("Resultados da avaliação salvos em avaliacao_modelos.npz e avaliacao_modelos.csv")


Treinando e avaliando KNN...
Treinando e avaliando MLP...
     Acurácia  Precision    Recall        F1
KNN    0.9696   0.894557  0.961500  0.926787
MLP    0.9538   0.890172  0.878474  0.883484
Resultados da avaliação salvos em avaliacao_modelos.npz e avaliacao_modelos.csv


In [ ]:
# Removido pela questão do metódo SVM (Support Vector Machine) ser lento que só uma porra e pior que o kNN (k-Nearest Neighbors)

# Configurações
# n_splits = 3
# random_state = 42

# def evaluate_model(model, X, y, kf):
#     accs, precs, recs, f1s = [], [], [], []
#     for train_idx, test_idx in kf.split(X, y):
#         model.fit(X[train_idx], y[train_idx])
#         preds = model.predict(X[test_idx])
#         accs.append(accuracy_score(y[test_idx], preds))
#         precs.append(precision_score(y[test_idx], preds))
#         recs.append(recall_score(y[test_idx], preds))
#         f1s.append(f1_score(y[test_idx], preds))
#     return np.mean(accs), np.mean(precs), np.mean(recs), np.mean(f1s)

# kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

# # Criar modelos
# model_knn = KNeighborsClassifier(n_neighbors=3)
# model_svm = SVC(kernel='linear')
# model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200)

# # Avaliar modelos
# results = {}
# for name, model in [("KNN", model_knn), ("SVM", model_svm), ("MLP", model_mlp)]:
#     print(f"Treinando e avaliando {name}...")
#     acc, prec, rec, f1 = evaluate_model(model, X, y, kf)
#     results[name] = (acc, prec, rec, f1)
#     # Salvar modelo final
#     dump(model, f"{name}_final_model.pkl")

# # Exibir resultados

# df = pd.DataFrame(results, index=["Acurácia", "Precision", "Recall", "F1"]).T
# print(df)

# # -----------------------------
# # Salvar resultados para futuras execuções
# # -----------------------------

# # Salvar em NPZ (Python)
# np.savez("avaliacao_modelos.npz", **results)

# # Salvar em CSV (legível)
# df.to_csv("avaliacao_modelos.csv")

# print("Resultados da avaliação salvos em avaliacao_modelos.npz e avaliacao_modelos.csv")


In [4]:
# Carregar resultados salvos
data = np.load("avaliacao_modelos.npz")
for model in data:
    acc, prec, rec, f1 = data[model]
    print(model, acc, prec, rec, f1)


KNN 0.9695997494170533 0.8945573861863337 0.9615004809907358 0.9267869728533845
MLP 0.9537996991060833 0.8901721118656515 0.8784744264504384 0.8834841083593457


# Melhor Modelo – Comparação

Após avaliar os modelos usando cross-validation (3 folds) sobre o dataset de vogais manuscritas para detectar a letra “i”, os resultados foram:

Primeira avaliação
Modelo  Acurácia    Precision   Recall	F1	    Observações
KNN	    0.9874	    0.9467	    0.9930	0.9693	Mais rápido, melhor F1, ideal para uso prático
MLP	    0.9788	    0.9673	    0.9255	0.9459	Boa precisão, mas treino mais lento
SVM	    0.9705	    0.9220	    0.9315	0.9267	demora (25+ minutos), e menor Score

Segunda avaliação
Modelo  Acurácia    Precision   Recall	F1	    Observações
KNN    0.9696   0.894557  0.961500  0.926787    Mais rápido, melhor F1, ideal para uso prático pelo menor tempo de treino
MLP    0.9538   0.890172  0.878474  0.883484    Boa precisão, treino semelhante em tempo porém menor F1